This notebook computes the derivative of the resilience indicator. This information is usefull for rendering the scorcards. This notebook must be run after compute_res_ind and before render_scorecards

In [30]:
%reset-f
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("always",category=UserWarning)
import os, time
import itertools

from progress_reporter import *
from res_ind_lib import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
#Folder where outputs are stored
out_folder = "scorecards/"
os.makedirs(out_folder,exist_ok=True)

In [32]:
pd.set_option('mode.use_inf_as_null', True)

# data

In [43]:
#Load the excel data file into a dataframe (table)
df_original=pd.read_excel("all_data_compiled.xlsx", 
                 index_col=0, #tells Pandas to use the first column (Provinces) as the index of the table
                 skiprows=[0, 2] #The first (0) row is a description of the variables and the 3rd (2) row is empty. We skip them
                ) 
df_original.index.name="province"
#Displays the first rows of that dataframe
df_original=def_ref_values(df_original)

#appends "countries" from the sensitivity analysis
# df_original=df_original.append(pd.read_csv("df_sensit_input.csv",index_col="Province"))

df_with_results=pd.read_csv("all_data_and_results.csv",index_col="province")

#Rankings 
ranks = df_with_results.dropna(how="all",axis=1).dropna().rank(method="min",ascending =False) 
ranks.to_csv("rankings.csv")

#appends sensitivity anlysis
# df_with_results=df_with_results.append(pd.read_csv("df_sensit_results.csv").set_index("Province"))

In [44]:
df_with_results.columns.values


array(['gdp_pc_pp', 'pop', 'pov_head', 'shew', 'gdp_pc_pp_nat',
       'rel_gdp_pp', 'share1', 'axfin_p', 'axfin_r', 'social_p',
       'social_r', 'fap', 'far', 'v_p', 'v_r', 'protection', 'avg_prod_k',
       'T_rebuild_K', 'pi', 'nat_buyout', 'sigma_r', 'sigma_p',
       'income_elast', 'rho', 'protectionref', 'gdp_pc_pp_ref', 'v_s',
       'dK', 'delta_W', 'dcap', 'dcar', 'dKtot', 'deltaW_nat',
       'equivalent_cost', 'risk', 'total_equivalent_cost', 'resilience',
       'risk_to_assets'], dtype=object)

# computes all derivatives

In [67]:
deriv_set = np.setdiff1d( df_original.columns,
    ["pop","iso3","original_country","income_elast","avg_prod_k","share1",
     "protectionref"])

pd.DataFrame(data=deriv_set).to_csv("inputs/deriv_set.csv",index=False,header=False)

score_card_set = ["risk_to_assets","risk"]
                  #"delta_W","dWsurWprime", "equivalent_cost", "dW_destitution", "total_equivalent_cost_no_destitution" ]

#Checks that info has information for all variables in deriv_set
info = pd.read_csv("inputs/inputs_info.csv").set_index("key")
for d in deriv_set:
    if d not in info.index.values:
        raise Exception(d+" is not documented in inputs_info.csv")

        
score_card_set

['risk_to_assets', 'risk']

In [48]:
def compute_derivative(df_original,deriv_set,score_card_set):
    der = pd.DataFrame(index=df_original.index, columns=pd.MultiIndex.from_product([deriv_set,score_card_set], names=['inputs', 'outputs']))
    h=0.0001
    #loop on all data in df prior to add the results
    fx = compute_resiliences(df_original)[score_card_set]
    for var in deriv_set:
        try:
            progress_reporter(var)
            df_=df_original.copy(deep=True)
            df_[var]=df_[var]+h
            fxh= compute_resiliences(df_)[score_card_set]
            der[var] = (fxh-fx)/(h)
        except TypeError:
            print("no derivative for " +var)
    return der.swaplevel('inputs', 'outputs', axis=1).sort_index(axis=1)

In [49]:
df_original.columns

Index(['gdp_pc_pp', 'pop', 'pov_head', 'shew', 'gdp_pc_pp_nat', 'rel_gdp_pp',
       'share1', 'axfin_p', 'axfin_r', 'social_p', 'social_r', 'fap', 'far',
       'v_p', 'v_r', 'protection', 'avg_prod_k', 'T_rebuild_K', 'pi',
       'nat_buyout', 'sigma_r', 'sigma_p', 'income_elast', 'rho',
       'protectionref', 'gdp_pc_pp_ref', 'v_s'],
      dtype='object')

In [52]:
derivatives = compute_derivative(df_original,deriv_set,score_card_set)

Currently working on:  v_s


# output

output signs in excel

In [64]:
#saves derivatives in excel tabs with signs in colors (very usefull for understanding the model)
writer= pd.ExcelWriter("results/signs.xlsx", engine='xlsxwriter')
workbook=writer.book
# Add a format. Light red fill with dark red text.
red = workbook.add_format({'bg_color': '#FFC7CE',
                               'font_color': '#9C0006'})

blue = workbook.add_format({'bg_color': '#92c5de',
                               'font_color': '#000061'})
for outname in score_card_set:
    sheetname = outname[0:30]
    #for outname in ["resilience"]:
    (derivatives[outname].dropna()).to_excel(writer,sheet_name=sheetname)
    writer.sheets[sheetname].conditional_format('B2:BB600', {'type':'cell',
                                    'criteria': '>',
                                    'value':    0,
                                    'format':   blue})
    writer.sheets[sheetname].conditional_format('B2:BB600', {'type':'cell',
                                    'criteria': '<',
                                    'value':    0,
                                    'format':   red})
    writer.sheets[sheetname].freeze_panes(1, 1)


In [65]:
try :
    writer.save()
except PermissionError:
    warnings.warn("Cannot write excel file. Check that it's not opened and try again")


In [66]:
#Signs of resilience derivative 
der =     np.sign(derivatives["risk"])
for i in signs.index:
    if (der[i].min()==der[i].max()): #all nonnan signs are equal
        signs[i]=der[i].min()
    else:
        print("ambigous sign for "+i)
        signs[i]=np.nan

